In [18]:
import pickle
ss = { "sp" : [1, 2, 3], "f0" : [4,5,6] }
dd = { "sp" : [1, 2, 3], "f0" : [4,5,6] }
d = { "val" : dd, "key" : ss }
afile = open(r'd.pkl', 'wb')

pickle.dump(d, afile)
afile.close()

In [19]:
print(d['val']['sp'])
foo = pickle.load(open("d.pkl", "rb"))
print(foo['val']['sp'])


[1, 2, 3]
[1, 2, 3]


In [36]:
import os
aiueo = ['a','i','u','e','o']
for x in os.walk('data'):
    for v in aiueo:
        path = os.path.join('data',x[0][5:],v+'.wav')
        print(path)


data\a.wav
data\i.wav
data\u.wav
data\e.wav
data\o.wav
data\我\a.wav
data\我\i.wav
data\我\u.wav
data\我\e.wav
data\我\o.wav
data\漢文\a.wav
data\漢文\i.wav
data\漢文\u.wav
data\漢文\e.wav
data\漢文\o.wav


In [ ]:
from termcolor import colored
import sys
import numpy as np
import os.path
import soundfile as sf
# import librosa
import pyworld as pw

#28 38 9 男
#28 38 9 女

frame_period = 10.0 # millisecond

aiueo = ['a','i','u','e','o']
for i in range(len(aiueo)):
    wavname = 'data/'+aiueo[i]+'.wav'
    if not os.path.isfile(wavname):
        print(colored('錯誤:找不到音檔','red'),wavname)
        continue

    x, fs = sf.read(wavname)
    x = (x[:,0] + x[:,1])/2 # stereo to mono

    num_of_frames = int(np.floor(len(x)/samples_per_frame))    
    f0, sp, ap = pw.wav2world(x, fs, frame_period =  frame_period)    # use default options


#   pitch judgement
    f0_note = 12*np.log2(f0[1:-1]/440)+49
    min_his = min(np.histogram(f0_note, bins=13)[0])